In [ ]:
!pip install arch
!pip install plotly
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 903 kB 25.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 25.8 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go 
from statsmodels.tsa.seasonal import seasonal_decompose
from arch.unitroot import ADF
from torchmetrics import SymmetricMeanAbsolutePercentageError
from google.colab import drive
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torchmetrics.functional import symmetric_mean_absolute_percentage_error

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
drive.mount('/content/gdrive')
path = "/content/gdrive/MyDrive/ML-Transportation-Rate/sample data/"

Mounted at /content/gdrive


In [ ]:
data = pd.read_csv(path + "TL_Cleaned_Sample_Data.csv")

In [ ]:
data.head()

,Unnamed: 0,SHIPMENT ID,CUSTOMER,DISTANCE,CASES,WEIGHT,VOLUME,SOURCE LOCATION ID,ORIGIN NAME,ORIGIN CITY,ORIGIN STATE,ORIGIN ZIP,DEST LOCATION ID,CONSIGNEE NAME,DEST CITY,DEST STATE,DEST ZIP,ACTUAL CARRIER,ACTUAL MODE,ACTUAL EQUIP,LINEHAUL COSTS,TOTAL ACTUAL COST,PU_APPT,DL_APPT,LINEHAUL UNIT COSTS,PC_MILER_PRACTICAL_MILEAGE,SPOT_AVG_LINEHAUL_RATE,SPOT_LOW_LINEHAUL_RATE,SPOT_HIGH_LINEHAUL_RATE,SPOT_FUEL_SURCHARGE,SPOT_TIME_FRAME,SPOT_ORIGIN_GEO_EXPANSION,SPOT_DESTINATION_GEO_EXPANSION,SPOT_NUMBER_OF_COMPANIES,SPOT_NUMBER_OF_REPORTS,SPOT_LINEHAUL_RATE_STDDEV,SPOT_YOUR_OWN_AVG_LINEHAUL_RATE,SPOT_YOUR_OWN_NUMBER_OF_REPORTS,SPOT_ERROR,CONTRACT_AVG_LINEHAUL_RATE,CONTRACT_LOW_LINEHAUL_RATE,CONTRACT_HIGH_LINEHAUL_RATE,CONTRACT_FUEL_SURCHARGE,CONTRACT_AVG_ACCESSORIAL_EXCLUDES_FUEL,CONTRACT_TIME_FRAME,CONTRACT_ORIGIN_GEO_EXPANSION,CONTRACT_DESTINATION_GEO_EXPANSION,CONTRACT_NUMBER_OF_COMPANIES,CONTRACT_NUMBER_OF_REPORTS,CONTRACT_LINEHAUL_RATE_STDDEV,CONTRACT_YOUR_OWN_AVG_LINEHAUL_RATE,CONTRACT_YOUR_OWN_NUMBER_OF_REPORTS,CONTRACT_ERROR,ORIG_CITY,ORIG_STATE,ORIG_POSTAL_CODE,DEST_CITY,DEST_STATE,DEST_POSTAL_CODE,TRUCK_TYPE,RUN_ID,SOURCE_FILE_NAME
0,106,MTNORAM.B22182M00994,5194020264032378374,18.6,1,16.0,5.0,3519848996290549827,9008742004616525841,LITITZ,PA,17543,5189889568954220467,3932306814741862562,LEBANON,PA,17042,-4513929889249989103,TL,53DV,350.0,363.25,7/12/2022 11:07,7/12/2022 13:07,18.817204,18.0,26.55,24.27,30.24,0.73,3.0,20-Mkt area,20-Mkt area,31.0,84.0,220.06,NaN,NaN,NaN,13.27,9.59,13.27,0.73,158.56,30.0,10-3-Digit Zip,20-Mkt area,3.0,283.0,1.09,NaN,NaN,NaN,NaN,PA,17543,NaN,PA,17042,V,660e1b5d-02b4-4939-9bde-d8cd9608c5a8,CI Rates 5-12.csv
1,107,MTNORAM.B22182M00994,5194020264032378374,18.6,1,16.0,5.0,3519848996290549827,9008742004616525841,LITITZ,PA,17543,5189889568954220467,3932306814741862562,LEBANON,PA,17042,-4513929889249989103,TL,53DV,350.0,363.25,7/12/2022 11:07,7/12/2022 13:07,18.817204,18.0,25.90,25.83,26.63,0.74,7.0,10-3-Digit Zip,20-Mkt area,4.0,14.0,6.25,NaN,NaN,NaN,24.41,10.36,29.40,0.74,158.02,30.0,20-Mkt area,20-Mkt area,39.0,1879.0,42.32,NaN,NaN,NaN,NaN,PA,17543,NaN,PA,17042,V,6fad52d5-d4bd-4c08-9cc1-2883ff4ddf92,CI Rate 6-15.csv
2,109,MTNORAM.B22182M00994,5194020264032378374,18.6,1,16.0,5.0,3519848996290549827,9008742004616525841,LITITZ,PA,17543,5189889568954220467,3932306814741862562,LEBANON,PA,17042,-4513929889249989103,TL,53DV,350.0,363.25,7/12/2022 11:07,7/12/2022 13:07,18.817204,18.0,24.31,23.92,25.83,0.74,7.0,10-3-Digit Zip,20-Mkt area,4.0,14.0,3.14,NaN,NaN,NaN,24.60,10.33,29.42,0.74,162.37,30.0,20-Mkt area,20-Mkt area,38.0,2032.0,50.68,NaN,NaN,NaN,NaN,PA,17543,NaN,PA,17042,V,b7984fdd-3c41-423a-b3a6-ef62a305d4dc,CI Rate 6-22.csv
3,115,EXEL.B22179E01740,-697198312127870698,334.2,1,24000.0,2500.0,-2030167911833778069,7396549228155826994,DUFFIELD,VA,24244,-8296507883433519476,5792972471462458170,LITHIA SPRINGS,GA,30122,6273882146989164462,TL,53DV,250.0,463.76,7/1/2022 20:00,7/5/2022 20:00,0.748055,328.0,3.49,2.83,5.02,0.74,3.0,20-Mkt area,20-Mkt area,11.0,16.0,0.90,NaN,NaN,NaN,4.64,3.93,5.58,0.74,158.02,30.0,10-3-Digit Zip,20-Mkt area,5.0,108.0,0.84,NaN,NaN,NaN,NaN,VA,24244,NaN,GA,30122,V,6fad52d5-d4bd-4c08-9cc1-2883ff4ddf92,CI Rate 6-15.csv
4,117,EXEL.B22179E01740,-697198312127870698,334.2,1,24000.0,2500.0,-2030167911833778069,7396549228155826994,DUFFIELD,VA,24244,-8296507883433519476,5792972471462458170,LITHIA SPRINGS,GA,30122,6273882146989164462,TL,53DV,250.0,463.76,7/1/2022 20:00,7/5/2022 20:00,0.748055,328.0,3.13,2.76,3.57,0.74,3.0,20-Mkt area,20-Mkt area,16.0,24.0,0.50,NaN,NaN,NaN,4.73,3.93,5.58,0.74,162.37,30.0,10-3-Digit Zip,20-Mkt area,4.0,90.0,0.87,NaN,NaN,NaN,NaN,VA,24244,NaN,GA,30122,V,b7984fdd-3c41-423a-b3a6-ef62a305d4dc,CI Rate 6-22.csv


In [ ]:
X_columns = ['DEST ZIP','DEST_POSTAL_CODE', 'DEST CITY', 'ORIGIN ZIP', 'ORIGIN CITY', 'ORIG_POSTAL_CODE', 'CUSTOMER', 'ACTUAL CARRIER', 'DEST LOCATION ID','VOLUME', 'DEST STATE', 'DEST_STATE','WEIGHT', 'ORIGIN STATE', 'ORIG_STATE', 'DISTANCE', 'ORIGIN NAME', 'CONTRACT_AVG_LINEHAUL_RATE', 'CONTRACT_HIGH_LINEHAUL_RATE','CASES']
dummies = ['DEST ZIP','DEST_POSTAL_CODE', 'DEST CITY', 'ORIGIN ZIP', 'ORIGIN CITY', 'ORIG_POSTAL_CODE', 'CUSTOMER', 'ACTUAL CARRIER', 'DEST LOCATION ID', 'DEST STATE', 'DEST_STATE', 'ORIGIN STATE', 'ORIG_STATE', 'ORIGIN NAME']

In [ ]:
first_data = data[X_columns + ['LINEHAUL COSTS']].copy()

In [ ]:
for col in dummies:
  temp_col = pd.get_dummies(first_data[col], prefix = col)
  first_data = pd.concat([first_data, temp_col],axis=1)
first_data.drop(dummies, axis=1, inplace=True)

In [ ]:
first_data = first_data.astype('float64')

In [ ]:
device = torch.device('cuda')

In [ ]:
# Build a LSTM Model def
def modeling(data, hidden_size, epoch = 10):
  # Step1: Determine tain data and test data. Spereate X columns and label ['LINEHAUL COSTS'] as Y columns
  X_train, X_test, y_train, y_test = train_test_split(data.drop('LINEHAUL COSTS',axis=1), data['LINEHAUL COSTS'], test_size=0.2, random_state=42)

  # Step 2: Load training X and training Yto the torch
  train_features = torch.tensor(X_train.values, dtype=torch.float).to(device)
  train_labels = torch.tensor(y_train.to_numpy(), dtype=torch.float).to(device)

  # Step 3: Load test X and test Y to the torch
  test_features = torch.tensor(X_test.values, dtype=torch.float).to(device)
  test_labels = torch.tensor(y_test.to_numpy(), dtype=torch.float).to(device)

  # Step 4: Print General information of trainging data and test data
  print("train data size: ", train_features.shape)
  print("label data size: ", train_labels.shape)
  print("test data size: ", test_features.shape)
  print("test label size: ", test_labels.shape)

  # Step 5: Step up model layer
  # Create two hidden layers
  # Setting ReLU as activation functions 
  my_nn = torch.nn.Sequential(
    torch.nn.Linear(train_features.shape[1], hidden_size),
    torch.nn.ReLU(),
    torch.nn.Linear(hidden_size, hidden_size),
    torch.nn.ReLU(),
    torch.nn.Linear(hidden_size, 1),
  ).to(device)

  # Step 6: Set SMAPE as accuracy measuratment
  cost = SymmetricMeanAbsolutePercentageError().to(device)

  # Step 7: Setup  ad learning rate as 0.001 and prepare a list to record losses
  optimizer = torch.optim.Adam(my_nn.parameters(), lr = 0.001)
  losses = []

  # Step 8: Setup learning iteration, the default learning iteration is 10
  for i in range(epoch):
      ## take train features and labels as copied tensor
      xx = torch.tensor(train_features, dtype = torch.float, requires_grad = True).to(device)
      yy = torch.tensor(train_labels, dtype = torch.float, requires_grad = True).to(device)

      ## use my_nn to predict and reshape to a 1D tensor
      prediction = my_nn(xx).to(device)
      prediction = torch.reshape(prediction,(prediction.shape[0],)).to(device)

      ## calculate cost using SMAPE
      loss = cost(prediction, yy)

      ## backward
      optimizer.zero_grad()
      loss.backward(retain_graph=True)
      optimizer.step()

      ## record loss
      losses.append(loss.data.cpu().numpy())

      if i % (epoch//10)==0:
          print(i, losses[len(losses)-1])

  # step 9: Find model test error
  pred = my_nn(test_features).to(device)
  pred = torch.reshape(pred, (pred.shape[0], ))
  loss = symmetric_mean_absolute_percentage_error(pred, test_labels)
  print("test:", loss)
  return my_nn

In [ ]:
model1 = modeling(first_data, 2500, 1000)

train data size:  torch.Size([13272, 3775])
label data size:  torch.Size([13272])
test data size:  torch.Size([3319, 3775])
test label size:  torch.Size([3319])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


0 1.9488988
100 0.66465336
200 0.47508544
300 0.43772826
400 0.39144838
500 0.38750827
600 0.33735245
700 0.31411996
800 0.3242988
900 0.30488455
test: tensor(0.3422, device='cuda:0', grad_fn=<DivBackward0>)


In [ ]:
torch.save(model1.state_dict(), '102801.pth')

In [ ]:
ltl_data = pd.read_csv(path + "LTL_Cleaned_Sample_Data.csv")

In [ ]:
X_columns = ['DEST ZIP','DEST_POSTAL_CODE', 'DEST CITY', 'ORIGIN ZIP', 'ORIGIN CITY', 'ORIG_POSTAL_CODE', 'CUSTOMER', 'ACTUAL CARRIER', 'DEST LOCATION ID','VOLUME', 'DEST STATE', 'DEST_STATE','WEIGHT', 'ORIGIN STATE', 'ORIG_STATE', 'DISTANCE', 'ORIGIN NAME', 'CONTRACT_AVG_LINEHAUL_RATE', 'CONTRACT_HIGH_LINEHAUL_RATE','CASES']
dummies = ['DEST ZIP','DEST_POSTAL_CODE', 'DEST CITY', 'ORIGIN ZIP', 'ORIGIN CITY', 'ORIG_POSTAL_CODE', 'CUSTOMER', 'ACTUAL CARRIER', 'DEST LOCATION ID', 'DEST STATE', 'DEST_STATE', 'ORIGIN STATE', 'ORIG_STATE', 'ORIGIN NAME']

In [ ]:
# More data clean work as we will classify dummy columns from X features
def clean_data(data, X_columns, dummies):
  first_data = data[X_columns + ['LINEHAUL COSTS']].copy()
  for col in dummies:
    temp_col = pd.get_dummies(first_data[col], prefix = col)
    first_data = pd.concat([first_data, temp_col],axis=1)
  first_data.drop(dummies, axis=1, inplace=True)
  first_data = first_data.astype('float64')
  return first_data

In [ ]:
ltl_cleaned = clean_data(ltl_data, X_columns, dummies)

In [ ]:
hidden_size = ltl_cleaned.shape[1]//3*2
hidden_size

4892

In [ ]:
model_ltl = modeling(ltl_cleaned, hidden_size, 1000)

train data size:  torch.Size([12016, 7337])
label data size:  torch.Size([12016])
test data size:  torch.Size([3005, 7337])
test label size:  torch.Size([3005])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


0 1.9167284
100 0.40807045
200 0.32442904
300 0.31523705
400 0.30279407
500 0.30519694
600 0.3108001
700 0.28935692
800 0.28316343
900 0.2707769
test: tensor(0.2784, device='cuda:0', grad_fn=<DivBackward0>)


In [ ]:
torch.save(model_ltl.state_dict(), "/content/gdrive/MyDrive/ML-Transportation-Rate/102701.pth")